In [ ]:
# ===============================
# E2E Agentic Resume–JD Matcher
# AutoGen | Demo from Sahil & CoPilot
# ===============================

from autogen import (
    AssistantAgent,
    UserProxyAgent,
    GroupChat,
    GroupChatManager
)
import os

# -------------------------------
# 1. LLM Configuration
# -------------------------------
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0,
    "api_key": os.environ.get("OPENAI_API_KEY")
}

# -------------------------------
# 2. Define Agents
# -------------------------------

resume_parser = AssistantAgent(
    name="ResumeParserAgent",
    llm_config=llm_config,
    system_message="""
    You extract structured information from resumes.
    Output JSON with:
    - skills
    - years_of_experience
    - roles
    """
)

jd_parser = AssistantAgent(
    name="JDParserAgent",
    llm_config=llm_config,
    system_message="""
    You extract structured requirements from job descriptions.
    Output JSON with:
    - required_skills
    - minimum_experience
    - role_level
    """
)

skill_matcher = AssistantAgent(
    name="SkillMatchAgent",
    llm_config=llm_config,
    system_message="""
    You compare resume skills with job required skills.
    Output:
    - matched_skills
    - missing_skills
    - skill_match_percentage
    """
)

experience_matcher = AssistantAgent(
    name="ExperienceMatchAgent",
    llm_config=llm_config,
    system_message="""
    You compare candidate experience with job requirements.
    Output:
    - experience_match (true/false)
    - explanation
    """
)

# reviewer = AssistantAgent(
#     name="ReviewerAgent",
#     llm_config=llm_config,
#     system_message="""
#     You are the final decision maker.
#     Combine all prior agent outputs.
#     Decide MATCH or NO MATCH.
#     Provide score (0–100) and reasoning.
#     End strictly with:

#     FINAL ANSWER:
#     """
#)

### Updated Reviewer Agent with clearer instructions
reviewer = AssistantAgent(
    name="ReviewerAgent",
    llm_config=llm_config,
    system_message="""
    You are the FINAL decision maker.

    Rules:
    - Read inputs from all other agents
    - Decide MATCH or NO MATCH
    - Provide reason and score
    - End your response EXACTLY with:

    FINAL ANSWER:
    <MATCH or NO MATCH>

    Do NOT ask questions.
    Do NOT continue the discussion.
    """
)

# -------------------------------
# 3. Group Chat Setup
# -------------------------------
group_chat = GroupChat(
    agents=[
        resume_parser,
        jd_parser,
        skill_matcher,
        experience_matcher,
        reviewer
    ],
    messages=[],
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
    max_round=10 ## This number can be tweeked
)

manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

# -------------------------------
# 4. User Proxy (Controls Flow)
# -------------------------------
# user = UserProxyAgent(
#     name="User",
#     human_input_mode="NEVER",
#     code_execution_config={
#         "use_docker": False
#     },
#     is_termination_msg=lambda msg: "FINAL ANSWER" in msg["content"]
# )

user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    is_termination_msg=lambda msg: (
        msg.get("name") == "ReviewerAgent"
        and "FINAL ANSWER" in msg.get("content", "")
    )
)

# -------------------------------
# 5. Resume + JD Input
# -------------------------------
resume_text = """
Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.
Worked on scalable backend systems.
"""

jd_text = """
Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.
"""

combined_input = f"""
RESUME:
{resume_text}

JOB DESCRIPTION:
{jd_text}

Evaluate if this candidate matches the job.
"""

# -------------------------------
# 6. Run the Agentic System
# -------------------------------
chat_result = user.initiate_chat(
    manager,
    message=combined_input
)

# -------------------------------
# 7. Extract FINAL ANSWER Safely
# -------------------------------
final_messages = [
    msg["content"]
    for msg in chat_result.chat_history
    if msg.get("name") == "ReviewerAgent"
]

final_output = final_messages[-1] if final_messages else "No FINAL ANSWER generated."

print("\n================ FINAL OUTPUT ================\n")
print(final_output)




User (to chat_manager):


RESUME:

Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.
Worked on scalable backend systems.


JOB DESCRIPTION:

Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.


Evaluate if this candidate matches the job.


--------------------------------------------------------------------------------

Next speaker: ResumeParserAgent

ResumeParserAgent (to chat_manager):

```json
{
  "skills": ["Python", "Django", "REST APIs", "PostgreSQL", "Docker"],
  "years_of_experience": 6,
  "roles": ["Senior Python Backend Engineer"],
  "matches_job": false,
  "reason": "The candidate does not have experience with AWS, which is a required skill for the job."
}
```

--------------------------------------------------------------------------------

Next speaker: JDParserAgent

JDParserAgent (to chat_manager):

```json
{
  "required_skills": ["Python", "

In [4]:
final_output

'No FINAL ANSWER generated.'

## With Final Answers

In [9]:
# ==========================================================
# AutoGen E2E Agentic Resume–JD Matcher
# FINAL ANSWER GUARANTEED
# Docker execution DISABLED
# ==========================================================

from autogen import (
    AssistantAgent,
    UserProxyAgent,
    GroupChat,
    GroupChatManager
)
import os

# ----------------------------------------------------------
# 1. LLM Configuration
# ----------------------------------------------------------
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0,
    "api_key": os.environ.get("OPENAI_API_KEY")
}

# ----------------------------------------------------------
# 2. Agents
# ----------------------------------------------------------
resume_parser = AssistantAgent(
    name="ResumeParserAgent",
    llm_config=llm_config,
    system_message="""
    Extract from resume:
    - skills
    - years_of_experience
    - roles
    """
)

jd_parser = AssistantAgent(
    name="JDParserAgent",
    llm_config=llm_config,
    system_message="""
    Extract from JD:
    - required_skills
    - minimum_experience
    - role_level
    """
)

skill_matcher = AssistantAgent(
    name="SkillMatchAgent",
    llm_config=llm_config,
    system_message="""
    Compare resume skills vs JD skills.
    Output matched_skills, missing_skills, percentage.
    """
)

experience_matcher = AssistantAgent(
    name="ExperienceMatchAgent",
    llm_config=llm_config,
    system_message="""
    Compare experience vs requirement.
    Output experience_match and explanation.
    """
)

reviewer = AssistantAgent(
    name="ReviewerAgent",
    llm_config=llm_config,
    system_message="""
    FINAL DECISION AGENT.

    Rules:
    - Read all prior outputs
    - Decide MATCH or NO MATCH
    - Give short reasoning
    - END EXACTLY WITH:

    FINAL ANSWER:
    <MATCH or NO MATCH>
    """
)

# ----------------------------------------------------------
# 3. GroupChat
# ----------------------------------------------------------
group_chat = GroupChat(
    agents=[
        resume_parser,
        jd_parser,
        skill_matcher,
        experience_matcher,
        reviewer
    ],
    messages=[],
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
    max_round=10
)

# ----------------------------------------------------------
# 4. Manager WITH TERMINATION LOGIC (KEY FIX)
# ----------------------------------------------------------
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
    is_termination_msg=lambda msg: (
        msg.get("name") == "ReviewerAgent"
        and "FINAL ANSWER" in msg.get("content", "")
    )
)

# ----------------------------------------------------------
# 5. User Proxy (Docker OFF)
# ----------------------------------------------------------
user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False}
)

# ----------------------------------------------------------
# 6. Input
# ----------------------------------------------------------
resume_text = """
Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.
"""

jd_text = """
Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.
"""

message = f"""
RESUME:
{resume_text}

JOB DESCRIPTION:
{jd_text}

Evaluate if this candidate matches the job.
"""

# ----------------------------------------------------------
# 7. Run
# ----------------------------------------------------------
chat_result = user.initiate_chat(
    manager,
    message=message
)

# ----------------------------------------------------------
# 8. Extract FINAL ANSWER (SAFE)
# ----------------------------------------------------------
final = [
    m["content"]
    for m in chat_result.chat_history
    if m.get("name") == "ReviewerAgent"
    and "FINAL ANSWER" in m.get("content", "")
][-1]

print("\n================ FINAL OUTPUT ================\n")
print(final)


User (to chat_manager):


RESUME:

Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.


JOB DESCRIPTION:

Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.


Evaluate if this candidate matches the job.


--------------------------------------------------------------------------------

Next speaker: ResumeParserAgent

ResumeParserAgent (to chat_manager):

The candidate matches the job description in the following ways:

- **Skills**: The candidate has the required skills of Python, Django, and REST APIs. However, they do not have experience with AWS, which is listed as a required skill in the job description.
  
- **Years of Experience**: The candidate has 6 years of experience, which meets the minimum requirement of 5 years.

- **Roles**: The candidate is a Senior Python Backend Engineer, which aligns with the job title of Senior Backend Engineer.

Overall, 

IndexError: list index out of range

In [8]:
final_output

'FINAL ANSWER NOT FOUND'

In [14]:
# ==========================================================
# AutoGen E2E Agentic Resume–JD Matcher
# FINAL ANSWER guaranteed
# Single Cell | Docker Disabled
# ==========================================================

from autogen import (
    AssistantAgent,
    UserProxyAgent,
    GroupChat,
    GroupChatManager
)
import os

# ----------------------------------------------------------
# 1. LLM Configuration
# ----------------------------------------------------------
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0,
    "api_key": os.environ.get("OPENAI_API_KEY")
}

# ----------------------------------------------------------
# 2. Define Agents (Strict Roles)
# ----------------------------------------------------------

resume_parser = AssistantAgent(
    name="ResumeParserAgent",
    llm_config=llm_config,
    system_message="""
    Extract from the resume:
    - skills
    - years_of_experience
    - roles

    Do NOT evaluate fit.
    """
)

jd_parser = AssistantAgent(
    name="JDParserAgent",
    llm_config=llm_config,
    system_message="""
    Extract from the job description:
    - required_skills
    - minimum_experience
    - role_level

    Do NOT evaluate fit.
    """
)

skill_matcher = AssistantAgent(
    name="SkillMatchAgent",
    llm_config=llm_config,
    system_message="""
    Compare resume skills with job required skills.

    Output:
    - matched_skills
    - missing_skills
    - skill_match_percentage

    Do NOT make the final decision.
    """
)

experience_matcher = AssistantAgent(
    name="ExperienceMatchAgent",
    llm_config=llm_config,
    system_message="""
    Compare candidate experience with job requirement.

    Output:
    - experience_match (true/false)
    - explanation

    Do NOT make the final decision.
    """
)

reviewer = AssistantAgent(
    name="ReviewerAgent",
    llm_config=llm_config,
    system_message="""
    You are the FINAL decision maker.

    RULES:
    - Read all prior agent outputs
    - Decide MATCH or NO MATCH
    - Provide a short justification
    - END your response EXACTLY with:

    FINAL ANSWER:
    <MATCH or NO MATCH>

    Do NOT continue discussion.
    Do NOT ask questions.
    """
)

# ----------------------------------------------------------
# 3. Group Chat Setup
# ----------------------------------------------------------
group_chat = GroupChat(
    agents=[
        resume_parser,
        jd_parser,
        skill_matcher,
        experience_matcher,
        reviewer
    ],
    messages=[],
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
    max_round=6
)

# ----------------------------------------------------------
# 4. GroupChatManager with TERMINATION LOGIC (KEY FIX)
# ----------------------------------------------------------
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
    is_termination_msg=lambda msg: (
        msg.get("name") == "ReviewerAgent"
        and "FINAL ANSWER" in msg.get("content", "")
    )
)

# ----------------------------------------------------------
# 5. User Proxy (Docker Disabled)
# ----------------------------------------------------------
user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False}
)

# ----------------------------------------------------------
# 6. Input: Resume + Job Description
# ----------------------------------------------------------
resume_text = """
Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.
"""

jd_text = """
Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.
"""

message = f"""
RESUME:
{resume_text}

JOB DESCRIPTION:
{jd_text}

Evaluate if this candidate matches the job.
"""

# ----------------------------------------------------------
# 7. Run the Agentic System
# ----------------------------------------------------------
chat_result = user.initiate_chat(
    manager,
    message=message
)

# ----------------------------------------------------------
# 8. Extract FINAL ANSWER (Deterministic)
# ----------------------------------------------------------
# final_answer = [
#     msg["content"]
#     for msg in chat_result.chat_history
#     if msg.get("name") == "ReviewerAgent"
#     and "FINAL ANSWER" in msg.get("content", "")
# ][-1]

# print("\n================ FINAL OUTPUT ================\n")
# print(final_answer)

## Fix List Index out of Range Error
# final_answer = next(
#     (m.get("content","") for m in reversed(chat_result.chat_history)
#      if m.get("name") == "ReviewerAgent" and "FINAL ANSWER" in m.get("content","")),
#     "FINAL ANSWER NOT FOUND"
# )

final_answer = None

for msg in chat_result.chat_history:
    sender = msg.get("name") or msg.get("sender") or ""
    text = msg.get("content") or ""

    if sender == "ReviewerAgent" and "FINAL ANSWER" in text:
        final_answer = text

if final_answer:
    print("\n================ FINAL OUTPUT ================\n")
    print(final_answer)
else:
    print("\n⚠️ FINAL ANSWER NOT FOUND")
    print("\n--- DEBUG: FULL CHAT HISTORY ---\n")
    for m in chat_result.chat_history:
        print(m)


User (to chat_manager):


RESUME:

Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.


JOB DESCRIPTION:

Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.


Evaluate if this candidate matches the job.


--------------------------------------------------------------------------------

Next speaker: ResumeParserAgent

ResumeParserAgent (to chat_manager):

- Skills: Python, Django, REST APIs, PostgreSQL, Docker
- Years of Experience: 6 years
- Roles: Senior Python Backend Engineer

--------------------------------------------------------------------------------

Next speaker: JDParserAgent

JDParserAgent (to chat_manager):

- required_skills: Python, Django, AWS, REST APIs
- minimum_experience: 5 years
- role_level: Senior Backend Engineer

--------------------------------------------------------------------------------

Next speaker: SkillMatchAgent

SkillMat

In [18]:
print("\n--- RAW CHAT HISTORY ---\n")
for msg in chat_result.chat_history:
    print(msg)



--- RAW CHAT HISTORY ---

{'content': '\nRESUME:\n\nSenior Python Backend Engineer with 6 years of experience.\nSkills: Python, Django, REST APIs, PostgreSQL, Docker.\n\n\nJOB DESCRIPTION:\n\nLooking for a Senior Backend Engineer.\nRequired Skills: Python, Django, AWS, REST APIs.\nMinimum Experience: 5 years.\n\n\nEvaluate if this candidate matches the job.\n', 'role': 'assistant', 'name': 'User'}


In [19]:
chat_result = user.initiate_chat(manager, message=message)

User (to chat_manager):


RESUME:

Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.


JOB DESCRIPTION:

Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.


Evaluate if this candidate matches the job.


--------------------------------------------------------------------------------

Next speaker: ResumeParserAgent

ResumeParserAgent (to chat_manager):

- Skills: Python, Django, REST APIs, PostgreSQL, Docker
- Years of Experience: 6 years
- Roles: Senior Python Backend Engineer

--------------------------------------------------------------------------------

Next speaker: JDParserAgent

JDParserAgent (to chat_manager):

- required_skills: Python, Django, AWS, REST APIs
- minimum_experience: 5 years
- role_level: Senior Backend Engineer

--------------------------------------------------------------------------------

Next speaker: SkillMatchAgent

SkillMat

In [20]:
final_answer = None

for msg in chat_result.chat_history:
    sender = msg.get("name") or msg.get("sender") or ""
    text = msg.get("content") or ""

    if sender == "ReviewerAgent" and "FINAL ANSWER" in text:
        final_answer = text

print("\nFINAL ANSWER:\n")
print(final_answer)


FINAL ANSWER:

None


In [21]:
# ==========================================================
# AutoGen E2E Resume–JD Matcher
# FINAL OUTPUT = STRICT JSON
# ==========================================================

from autogen import (
    AssistantAgent,
    UserProxyAgent,
    GroupChat,
    GroupChatManager
)
import os
import json

# ----------------------------------------------------------
# 1. LLM Configuration
# ----------------------------------------------------------
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0,
    "api_key": os.environ.get("OPENAI_API_KEY")
}

# ----------------------------------------------------------
# 2. Define Agents (Bounded Responsibilities)
# ----------------------------------------------------------

resume_parser = AssistantAgent(
    name="ResumeParserAgent",
    llm_config=llm_config,
    system_message="""
    Extract from the resume:
    - skills
    - years_of_experience
    - roles

    Output in plain text bullets.
    Do NOT evaluate fit.
    """
)

jd_parser = AssistantAgent(
    name="JDParserAgent",
    llm_config=llm_config,
    system_message="""
    Extract from the job description:
    - required_skills
    - minimum_experience
    - role_level

    Output in plain text bullets.
    Do NOT evaluate fit.
    """
)

skill_matcher = AssistantAgent(
    name="SkillMatchAgent",
    llm_config=llm_config,
    system_message="""
    Compare resume skills with job required skills.

    Output:
    - matched_skills
    - missing_skills
    - skill_match_percentage

    Do NOT make the final decision.
    """
)

experience_matcher = AssistantAgent(
    name="ExperienceMatchAgent",
    llm_config=llm_config,
    system_message="""
    Compare candidate experience with job requirement.

    Output:
    - experience_match (true/false)
    - explanation

    Do NOT make the final decision.
    """
)

# 🔐 STRICT JSON OUTPUT AGENT
reviewer = AssistantAgent(
    name="ReviewerAgent",
    llm_config=llm_config,
    system_message="""
    You are the FINAL decision maker.

    You MUST output ONLY valid JSON.
    DO NOT add markdown, explanations, or extra text.

    STRICT JSON SCHEMA:
    {
      "decision": "MATCH" or "NO MATCH",
      "score": integer between 0 and 100,
      "reason": string
    }

    RULES:
    - decision MUST be exactly "MATCH" or "NO MATCH"
    - score MUST be an integer
    - reason MUST be concise
    - Output NOTHING except the JSON object
    """
)

# ----------------------------------------------------------
# 3. Group Chat Setup
# ----------------------------------------------------------
group_chat = GroupChat(
    agents=[
        resume_parser,
        jd_parser,
        skill_matcher,
        experience_matcher,
        reviewer
    ],
    messages=[],
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False,
    max_round=6
)

# ----------------------------------------------------------
# 4. GroupChatManager (Terminate on JSON)
# ----------------------------------------------------------
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
    is_termination_msg=lambda msg: (
        msg.get("content", "").strip().startswith("{")
    )
)

# ----------------------------------------------------------
# 5. User Proxy (Docker Disabled)
# ----------------------------------------------------------
user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False}
)

# ----------------------------------------------------------
# 6. Input: Resume + Job Description
# ----------------------------------------------------------
resume_text = """
Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.
"""

jd_text = """
Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.
"""

message = f"""
RESUME:
{resume_text}

JOB DESCRIPTION:
{jd_text}

Evaluate if this candidate matches the job.
"""

# ----------------------------------------------------------
# 7. Run the Agentic System
# ----------------------------------------------------------
chat_result = user.initiate_chat(
    manager,
    message=message
)

# ----------------------------------------------------------
# 8. SAFE JSON EXTRACTION (100% RELIABLE)
# ----------------------------------------------------------
final_result = None

for msg in chat_result.chat_history:
    text = msg.get("content", "").strip()
    if text.startswith("{") and text.endswith("}"):
        try:
            final_result = json.loads(text)
        except json.JSONDecodeError:
            pass

print("\n================ FINAL JSON OUTPUT ================\n")
print(final_result)


User (to chat_manager):


RESUME:

Senior Python Backend Engineer with 6 years of experience.
Skills: Python, Django, REST APIs, PostgreSQL, Docker.


JOB DESCRIPTION:

Looking for a Senior Backend Engineer.
Required Skills: Python, Django, AWS, REST APIs.
Minimum Experience: 5 years.


Evaluate if this candidate matches the job.


--------------------------------------------------------------------------------

Next speaker: ResumeParserAgent

ResumeParserAgent (to chat_manager):

- Skills: Python, Django, REST APIs, PostgreSQL, Docker
- Years of Experience: 6 years
- Roles: Senior Python Backend Engineer

--------------------------------------------------------------------------------

Next speaker: JDParserAgent

JDParserAgent (to chat_manager):

- Required Skills: Python, Django, AWS, REST APIs
- Minimum Experience: 5 years
- Role Level: Senior Backend Engineer

--------------------------------------------------------------------------------

Next speaker: SkillMatchAgent

SkillMat